# IMPORTS

In [1]:
!pip install transformers

     |████████████████████████████████| 2.3MB 6.8MB/s 
     |████████████████████████████████| 3.3MB 31.0MB/s 
     |████████████████████████████████| 901kB 36.5MB/s 


In [2]:
#For camenbert
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 5.0MB/s 


In [3]:
!pip install google_trans_new

In [4]:
# Load Huggingface transformers
from transformers import TFBertModel,  BertConfig, BertTokenizerFast, TFCamembertModel, CamembertTokenizerFast, CamembertConfig,CamembertTokenizer
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Input, Dropout, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy
import os
import torch
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from google_trans_new import google_translator
translator = google_translator()

# RUN IF COLAB TPU IS USED

When TPU is used, Use strategy.scope() for training the model

In [5]:
assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

if 'COLAB_TPU_ADDR' in os.environ:
  TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
else:
  TF_MASTER=''

tpu_address = TF_MASTER

In [6]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))
print("Number of devices: ", len(tf.config.list_logical_devices('TPU')))

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.125.106.58:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.125.106.58:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]
Number of devices:  8


In [7]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Preprocessing

In [ ]:
#Loading the cases
path_to_json = '{PATH_TO_CASES_FOLDER}'
dataset= pd.DataFrame()
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

for json_file in json_files:
    json_file=path_to_json+json_file
    with open(json_file) as train_file:
        dict_train = json.load(train_file)
    file_df=pd.DataFrame.from_dict(pd.json_normalize(dict_train), orient='columns')
    dataset=dataset.append(file_df,ignore_index = True)

In [ ]:
#Drop unnessecary columns
dataset.drop(['client_reference','mail_subject','party_id','party_type','unit_id','organization','from_mail','comments'], axis=1, inplace=True) 

In [ ]:
#Text preprocessing
def text_processing(content):
    text_no_html = BeautifulSoup(str(content),"html.parser" ).get_text()
    text_alpha_chars = re.sub("[^a-zA-Z']", " ", str(text_no_html)) 
    text_lower = text_alpha_chars.lower()
    stops = set(stopwords.words('dutch') +stopwords.words('french')+stopwords.words('english'))
    text_no_stop_words = ' '
    for w in text_lower.split():
        if w not in stops:  
            text_no_stop_words = text_no_stop_words + w + ' '

    text_no_short_words=''
    for w in text_no_stop_words.split(): 
        if len(w) >=2:
            text_no_short_words = text_no_short_words + w + ' '

    text_no_eng_words=''
    for w in text_no_short_words.split():
        if w not in words.words():
          if w not in wordnet.words():
            text_no_eng_words = text_no_eng_words + w + ' '

    print(text_no_eng_words)

    return text_no_eng_words
#Get filenames from attachments
def get_fileNames(attachment):
    fileNames=[]
    for obj in attachment:
        fileNames.append(obj['filename'].replace('.pdf', ''))
        fileNamesStr = ' '.join(fileNames)
    return fileNamesStr
#Get content from attachments
def get_content(attachment):
    contents=[]
    for obj in attachment:
        contents.append(obj['content'])
        contentsStr = ' '.join(contents)
        contentsStr=text_processing(contentsStr)
    return contentsStr
    

In [ ]:
dataset['mail_content']=dataset['mail_content'].apply(text_processing)
dataset['attachments_filename']=dataset['attachments'].apply(get_fileNames)
dataset['attachments_content']=dataset['attachments'].apply(get_content)
dataset.drop('attachments', axis=1, inplace=True)

In [ ]:
#Only use top 6 of sub_case_type and damage_type
top_amount = 6
list_top_label_sub = list(dataset["labels.sub_case_type"].value_counts()[:top_amount].index.tolist())
print(list_top_label_sub)
dataset = dataset[dataset["labels.sub_case_type"].isin(list_top_label_sub)]
list_top_label_damage = list(dataset["labels.damage_type"].value_counts()[:top_amount].index.tolist())
print(list_top_label_damage)
dataset = dataset[dataset["labels.damage_type"].isin(list_top_label_damage)]

In [ ]:
#1 input
dataset['content']=dataset[dataset.columns[1:]].apply(lambda x: ' '.join(x.dropna().astype(str)),
    axis=1)

In [ ]:
#1 input
dataset=dataset.drop(['attachments_filename','attachments_content','mail_content'],axis=1)

In [ ]:
#Create thumbling window for the attachment_content text. Limit of 500 words on LSTM
n=500
dataset=dataset.set_index(['language','labels.assignment_type','labels.case_type','labels.sub_case_type','labels.damage_type'])['content'].str.split().apply(
               lambda x: pd.Series([' '.join(x[i:i+n]) for i in range(0, len(x), n)])
            ).stack().reset_index()
dataset.columns = [*dataset.columns[:-1], 'content']

In [ ]:
dataset

In [ ]:
#drop level column ex: level_7
dataset=dataset.drop(['level_NUMBER'],axis=1)

In [ ]:
#SPLIT NL AND FR
dataset_nl=dataset.loc[dataset['language']=='nl_BE']
dataset_fr=dataset.loc[dataset['language']=='fr_BE']
dataset_nl.drop('language', axis=1, inplace=True) 
dataset_fr.drop('language', axis=1, inplace=True)

In [ ]:
def translate_text_nl(text):
  textEN= translator.translate(text,lang_tgt='en')
  textNL=translator.translate(textEN,lang_tgt='nl')
  return textNL

def translate_text_fr(text):
  textEN= translator.translate(text,lang_tgt='en')
  textFR=translator.translate(textEN,lang_tgt='fr')
  return textFR

In [ ]:
dataset_nl['content'] = dataset_nl['content'].apply(translate_text_nl)
dataset_fr['content'] = dataset_fr['content'].apply(translate_text_fr)

In [11]:
#NL
#Split targets and features
y_nl=dataset_nl[['labels.case_type','labels.sub_case_type','labels.damage_type','labels.assignment_type']]
X_nl=dataset_nl.drop(['labels.case_type','labels.sub_case_type','labels.damage_type','labels.assignment_type'], axis=1)

In [12]:
#FR
#Split targets and features
y_fr=dataset_fr[['labels.case_type','labels.sub_case_type','labels.damage_type','labels.assignment_type']]
X_fr=dataset_fr.drop(['labels.case_type','labels.sub_case_type','labels.damage_type','labels.assignment_type'], axis=1)

In [13]:
#NL
#Split train and test data
X_train_nl, X_test_nl, y_train_nl, y_test_nl = train_test_split(X_nl,y_nl, test_size = 0.20, random_state=0)
print(X_train_nl.shape,y_train_nl.shape)
print(X_test_nl.shape,y_test_nl.shape)

(5320, 1) (5320, 4)
(1330, 1) (1330, 4)


In [14]:
#FR
#Split train and test data
X_train_fr, X_test_fr, y_train_fr, y_test_fr = train_test_split(X_fr,y_fr, test_size = 0.20, random_state=0,stratify=y_fr['labels.case_type'])
print(X_train_fr.shape,y_train_fr.shape)
print(X_test_fr.shape,y_test_fr.shape)

(2342, 1) (2342, 4)
(586, 1) (586, 4)


In [15]:
#Safe original test set(for evaluation)
#NL
X_test_original_nl=X_test_nl
#FR
X_test_original_fr=X_test_fr

# Tokenizer

In [16]:
#BERT NL
with strategy.scope():
  model_name = 'wietsedv/bert-base-dutch-cased'
  # Load transformers config
  config = BertConfig.from_pretrained(model_name)
  config.output_hidden_states = False
  # Load BERT tokenizer
  tokenizer_nl = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
  # Load the Transformers BERT model
  transformer_model_nl = TFBertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at wietsedv/bert-base-dutch-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at wietsedv/bert-base-dutch-cased and are newly initialized: ['bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
#BERT FR
with strategy.scope():
  model_name = 'jplu/tf-camembert-base'
  # Load transformers config and set output_hidden_states to False
  config = CamembertConfig.from_pretrained(model_name)
  config.output_hidden_states = False
  # Load BERT tokenizer
  tokenizer_fr = CamembertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
  # Load the Transformers BERT model
  transformer_model_fr = TFCamembertModel.from_pretrained(model_name, config = config)

Some layers from the model checkpoint at jplu/tf-camembert-base were not used when initializing TFCamembertModel: ['lm_head']
- This IS expected if you are initializing TFCamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFCamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFCamembertModel were initialized from the model checkpoint at jplu/tf-camembert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertModel for predictions without further training.


In [18]:
#NL
def tokenize_content_nl(content,max_length):
    return tokenizer_nl(
    text=content.to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

#FR
def tokenize_content_fr(content,max_length):
    return tokenizer_fr(
    text=content.to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [19]:
#1 input NL
X_train_nl=tokenize_content_nl(X_train_nl['content'],512)
X_test_nl=tokenize_content_nl(X_test_nl['content'],512)

In [20]:
#1 input FR
X_train_fr=tokenize_content_fr(X_train_fr['content'],512)
X_test_fr=tokenize_content_fr(X_test_fr['content'],512)

In [21]:
#Categorical NL
#assignment_type
train_assignment_cats_nl=pd.Categorical(y_train_nl['labels.assignment_type'])
test_assignment_cats_nl=pd.Categorical(y_test_nl['labels.assignment_type'])
y1_train_nl=train_assignment_cats_nl.codes
y1_test_nl=test_assignment_cats_nl.codes

#case_type
train_case_cats_nl=pd.Categorical(y_train_nl['labels.case_type'])
test_case_cats_nl=pd.Categorical(y_test_nl['labels.case_type'])
y2_train_nl=train_case_cats_nl.codes
y2_test_nl=test_case_cats_nl.codes

#sub_case_type
train_sub_case_cats_nl=pd.Categorical(y_train_nl['labels.sub_case_type'])
test_sub_case_cats_nl=pd.Categorical(y_test_nl['labels.sub_case_type'])
y3_train_nl=train_sub_case_cats_nl.codes
y3_test_nl=test_sub_case_cats_nl.codes

#damage_type
train_damage_cats_nl=pd.Categorical(y_train_nl['labels.damage_type'])
test_damage_cats_nl=pd.Categorical(y_test_nl['labels.damage_type'])
y4_train_nl=train_damage_cats_nl.codes
y4_test_nl=test_damage_cats_nl.codes

print('Shape of label tensor Y1 Train:', y1_train_nl.shape)
print('Shape of label tensor Y1 Test:', y1_test_nl.shape)
print('Shape of label tensor Y2 Train:', y2_train_nl.shape)
print('Shape of label tensor Y2 Test:', y2_test_nl.shape)
print('Shape of label tensor Y3 Train:', y3_train_nl.shape)
print('Shape of label tensor Y3 Test:', y3_test_nl.shape)
print('Shape of label tensor Y4 Train:', y4_train_nl.shape)
print('Shape of label tensor Y4 Test:', y4_test_nl.shape)

Shape of label tensor Y1 Train: (5320,)
Shape of label tensor Y1 Test: (1330,)
Shape of label tensor Y2 Train: (5320,)
Shape of label tensor Y2 Test: (1330,)
Shape of label tensor Y3 Train: (5320,)
Shape of label tensor Y3 Test: (1330,)
Shape of label tensor Y4 Train: (5320,)
Shape of label tensor Y4 Test: (1330,)


In [22]:
#Categorical FR
#assignment_type
train_assignment_cats_fr=pd.Categorical(y_train_fr['labels.assignment_type'])
test_assignment_cats_fr=pd.Categorical(y_test_fr['labels.assignment_type'])
y1_train_fr=train_assignment_cats_fr.codes
y1_test_fr=test_assignment_cats_fr.codes

#case_type
train_case_cats_fr=pd.Categorical(y_train_fr['labels.case_type'])
test_case_cats_fr=pd.Categorical(y_test_fr['labels.case_type'])
y2_train_fr=train_case_cats_fr.codes
y2_test_fr=test_case_cats_fr.codes

#sub_case_type
train_sub_case_cats_fr=pd.Categorical(y_train_fr['labels.sub_case_type'])
test_sub_case_cats_fr=pd.Categorical(y_test_fr['labels.sub_case_type'])
y3_train_fr=train_sub_case_cats_fr.codes
y3_test_fr=test_sub_case_cats_fr.codes

#damage_type
train_damage_cats_fr=pd.Categorical(y_train_fr['labels.damage_type'])
test_damage_cats_fr=pd.Categorical(y_test_fr['labels.damage_type'])
y4_train_fr=train_damage_cats_fr.codes
y4_test_fr=test_damage_cats_fr.codes

print('Shape of label tensor Y1 Train:', y1_train_fr.shape)
print('Shape of label tensor Y1 Test:', y1_test_fr.shape)
print('Shape of label tensor Y2 Train:', y2_train_fr.shape)
print('Shape of label tensor Y2 Test:', y2_test_fr.shape)
print('Shape of label tensor Y3 Train:', y3_train_fr.shape)
print('Shape of label tensor Y3 Test:', y3_test_fr.shape)
print('Shape of label tensor Y4 Train:', y4_train_fr.shape)
print('Shape of label tensor Y4 Test:', y4_test_fr.shape)

Shape of label tensor Y1 Train: (2342,)
Shape of label tensor Y1 Test: (586,)
Shape of label tensor Y2 Train: (2342,)
Shape of label tensor Y2 Test: (586,)
Shape of label tensor Y3 Train: (2342,)
Shape of label tensor Y3 Test: (586,)
Shape of label tensor Y4 Train: (2342,)
Shape of label tensor Y4 Test: (586,)


# Models

## NL

In [ ]:
#1 input NL
with strategy.scope():
  # Load the MainLayer
  bert_nl = transformer_model_nl.layers[0]
  #Optimizer
  opt = tf.keras.optimizers.Adam(lr=0.0000001)
  # Early stopping
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=35)

  #checkpoint
  checkpoint_path='/content/drive/MyDrive/BERT_bestmodel.h5'
  mc=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True)

  #Model
  input_ids = Input(shape=(512),dtype='int32',name='input_ids')
  attention_mask = Input(shape=(512,), name='attention_mask', dtype='int32') 
  inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

  bert_model_nl=bert_nl(inputs)[1]

  dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
  pooled_output = dropout(bert_model_nl, training=False)
  d=Dropout(0.5)(pooled_output)

  
  g=Dense(64, activation='gelu')(d)

  #Output layers
  o1=Dense(units=2, activation='sigmoid', name='assignment_type')(g)
  o2=Dense(units=2, activation='sigmoid', name='case_type')(g)
  o3=Dense(units=6, activation='softmax', name='sub_case_type')(g)
  o4=Dense(units=6, activation='softmax', name='damage_type')(g)

  epochs = 300
  batch_size = 64

  model_nl = Model(inputs=[inputs], outputs=[o1, o2, o3, o4])
  model_nl.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model_nl.summary()


history = model_nl.fit(x={'input_ids':X_train_nl['input_ids'], 'attention_mask': X_train_nl['attention_mask']},y=[y1_train_nl,y2_train_nl,y3_train_nl,y4_train_nl],epochs=epochs, batch_size=batch_size,validation_split=0.2,callbacks=[es,mc])

## FR

In [ ]:
#1 input FR
with strategy.scope():
  # Load the MainLayer
  bert_fr = transformer_model_fr.layers[0]
  #Optimizer
  opt = tf.keras.optimizers.Adam(lr=0.0000001)
  # Early stopping
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=35)

  #checkpoint
  checkpoint_path='/content/drive/MyDrive/BERT__FR_bestmodel.h5'
  mc=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True)

  #Model
  input_ids = Input(shape=(512),dtype='int32',name='input_ids')
  attention_mask = Input(shape=(512,), name='attention_mask', dtype='int32') 
  inputs = {'input_ids': input_ids, 'attention_mask': attention_mask}

  bert_model_fr=bert_fr(inputs)[1]

  dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
  pooled_output = dropout(bert_model_fr, training=False)
  d=Dropout(0.5)(pooled_output)
  r=Dense(64, activation='gelu')(d)
  o1=Dense(units=2, activation='sigmoid', name='assignment_type')(r)
  o2=Dense(units=3, activation='softmax', name='case_type')(r)
  o3=Dense(units=6, activation='softmax', name='sub_case_type')(r)
  o4=Dense(units=6, activation='softmax', name='damage_type')(r)

  epochs = 300
  batch_size = 64

  model_fr = Model(inputs=[inputs], outputs=[o1, o2, o3, o4])
  model_fr.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model_fr.summary()

history = model_fr.fit(x={'input_ids':X_train_fr['input_ids'], 'attention_mask': X_train_fr['attention_mask']},y={'assignment_type':y1_train_fr,'case_type':y2_train_fr,'sub_case_type':y3_train_fr,'damage_type':y4_train_fr},epochs=epochs, batch_size=batch_size,validation_split=0.2,callbacks=[es,mc])

# Evaluation

In [ ]:
score = model.evaluate(x=[X_test_mc,X_test_af,X_test_ac], y=[y1_test, y2_test, y3_test, y4_test], verbose=1)
print("Loss:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('assignment_type_accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.plot(history.history['assignment_type_accuracy'], label='train')
plt.plot(history.history['val_assignment_type_accuracy'], label='test')
plt.legend()
plt.show();

plt.title('damage_type_accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.plot(history.history['damage_type_accuracy'], label='train')
plt.plot(history.history['val_damage_type_accuracy'], label='test')
plt.legend()
plt.show();

plt.title('case_type_accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.plot(history.history['case_type_accuracy'], label='train')
plt.plot(history.history['val_case_type_accuracy'], label='test')
plt.legend()
plt.show();

plt.title('sub_case_type_accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.plot(history.history['sub_case_type_accuracy'], label='train')
plt.plot(history.history['val_sub_case_type_accuracy'], label='test')
plt.legend()
plt.show();

In [ ]:
plt.title('assignment_type_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.plot(history.history['assignment_type_loss'], label='train')
plt.plot(history.history['val_assignment_type_loss'], label='test')
plt.legend()
plt.show();

plt.title('damage_type_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.plot(history.history['damage_type_loss'], label='train')
plt.plot(history.history['val_damage_type_loss'], label='test')
plt.legend()
plt.show();

plt.title('case_type_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.plot(history.history['case_type_loss'], label='train')
plt.plot(history.history['val_case_type_loss'], label='test')
plt.legend()
plt.show();

plt.title('sub_case_type_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.plot(history.history['sub_case_type_loss'], label='train')
plt.plot(history.history['val_sub_case_type_loss'], label='test')
plt.legend()
plt.show();

# Test predictions

In [ ]:
with strategy.scope():
  y1_pred_nl,y2_pred_nl,y3_pred_nl,y4_pred_nl = model_nl.predict({'input_ids':X_test_nl['input_ids'],'attention_mask': X_test_nl['attention_mask']})

In [ ]:
with strategy.scope():
  y1_pred_fr,y2_pred_fr,y3_pred_fr,y4_pred_fr = model_fr.predict({'input_ids':X_test_fr['input_ids'],'attention_mask': X_test_fr['attention_mask']})

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond_8/Identity:0' shape=(None, 512) dtype=int32>, <tf.Tensor 'cond_8/Identity_1:0' shape=(None, 512) dtype=int32>]


In [ ]:
#Dataframe NL
predictions_assign_nl=pd.DataFrame(columns=['pred.assignment'],data=test_assignment_cats_nl.categories[np.argmax(y1_pred_nl,axis=1)])
predictions_case_nl=pd.DataFrame(columns=['pred.case'],data=test_case_cats_nl.categories[np.argmax(y2_pred_nl,axis=1)])
predictions_sub_case_nl=pd.DataFrame(columns=['pred.sub_case'],data=test_sub_case_cats_nl.categories[np.argmax(y3_pred_nl,axis=1)])
predictions_damage_nl=pd.DataFrame(columns=['pred.damage'],data=test_damage_cats_nl.categories[np.argmax(y4_pred_nl,axis=1)])
result_nl = pd.concat([X_test_original_nl,predictions_case_nl,predictions_sub_case_nl,predictions_damage_nl,predictions_assign_nl,y_test_nl], axis=1, join="inner")

In [ ]:
#Dataframe FR
predictions_assign_fr=pd.DataFrame(columns=['pred.assignment'],data=test_assignment_cats_fr.categories[np.argmax(y1_pred_fr,axis=1)])
predictions_case_fr=pd.DataFrame(columns=['pred.case'],data=test_case_cats_fr.categories[np.argmax(y2_pred_fr,axis=1)])
predictions_sub_case_fr=pd.DataFrame(columns=['pred.sub_case'],data=test_sub_case_cats_fr.categories[np.argmax(y3_pred_fr,axis=1)])
predictions_damage_fr=pd.DataFrame(columns=['pred.damage'],data=test_damage_cats_fr.categories[np.argmax(y4_pred_fr,axis=1)])
result_fr = pd.concat([X_test_original_fr,predictions_case_fr,predictions_sub_case_fr,predictions_damage_fr,predictions_assign_fr,y_test_fr], axis=1, join="inner")

In [ ]:
#NL
result_nl

In [ ]:
#FR
result_fr

# Test Classification Reports

In [ ]:
#Categorical NL
cf = confusion_matrix(test_assignment_cats_nl,test_assignment_cats_nl.categories[np.argmax(y1_pred_nl,axis=1)])
cr= classification_report(test_assignment_cats_nl,test_assignment_cats_nl.categories[np.argmax(y1_pred_nl,axis=1)],target_names=pd.Categorical(test_assignment_cats_nl.categories))
print("assignment_type NL")
print(cr)
print(cf)

In [ ]:
#Categorical FR
cf = confusion_matrix(test_assignment_cats_fr,test_assignment_cats_fr.categories[np.argmax(y1_pred_fr,axis=1)])
cr= classification_report(test_assignment_cats_fr,test_assignment_cats_nl.categories[np.argmax(y1_pred_fr,axis=1)],target_names=pd.Categorical(test_assignment_cats_fr.categories))
print("assignment_type FR")
print(cr)
print(cf)

In [ ]:
#Categorical NL
cf = confusion_matrix(test_case_cats_nl,test_case_cats_nl.categories[np.argmax(y2_pred_nl,axis=1)])
cr= classification_report(test_case_cats_nl,test_case_cats_nl.categories[np.argmax(y2_pred_nl,axis=1)],target_names=pd.Categorical(test_case_cats_nl.categories))
print("case_type NL")
print(cr)
print(cf)

In [ ]:
#Categorical FR
cf = confusion_matrix(test_case_cats_fr,test_case_cats_fr.categories[np.argmax(y2_pred_fr,axis=1)])
cr= classification_report(test_case_cats_fr,test_case_cats_fr.categories[np.argmax(y2_pred_fr,axis=1)],target_names=pd.Categorical(test_case_cats_fr.categories))
print("case_type FR")
print(cr)
print(cf)

In [ ]:
#Categorical NL
cf = confusion_matrix(test_sub_case_cats_nl,test_sub_case_cats_nl.categories[np.argmax(y3_pred_nl,axis=1)])
cr= classification_report(test_sub_case_cats_nl,test_sub_case_cats_nl.categories[np.argmax(y3_pred_nl,axis=1)],target_names=pd.Categorical(test_sub_case_cats_nl.categories))
print("sub_case_type NL")
print(cr)
print(cf)

In [ ]:
#Categorical FR
cf = confusion_matrix(test_sub_case_cats_fr,test_sub_case_cats_fr.categories[np.argmax(y3_pred_fr,axis=1)])
cr= classification_report(test_sub_case_cats_fr,test_sub_case_cats_fr.categories[np.argmax(y3_pred_fr,axis=1)],target_names=pd.Categorical(test_sub_case_cats_fr.categories))
print("sub_case_type FR")
print(cr)
print(cf)

In [ ]:
#Categorical NL
cf = confusion_matrix(test_damage_cats_nl,test_damage_cats_nl.categories[np.argmax(y4_pred_nl,axis=1)])
cr= classification_report(test_damage_cats_nl,test_damage_cats_nl.categories[np.argmax(y4_pred_nl,axis=1)],target_names=pd.Categorical(test_damage_cats_nl.categories))
print("damage_type")
print(cr)
print(cf)

In [ ]:
#Categorical FR
cf = confusion_matrix(test_damage_cats_fr,test_damage_cats_fr.categories[np.argmax(y4_pred_fr,axis=1)])
cr= classification_report(test_damage_cats_fr,test_damage_cats_fr.categories[np.argmax(y4_pred_fr,axis=1)],target_names=pd.Categorical(test_damage_cats_fr.categories))
print("damage_type")
print(cr)
print(cf)